In [1]:
import pandas as pd
import numpy as np
import json
import os
import re

In [7]:
def create_df():
    """
    parse Json to get unspecified field count
    
    Args:
        data (string): String of properly formatted JSON.

    Returns:
        
    """
    dictionary = {}
    path_sanitized_policies = "/Users/bhramoss/Documents/CMU/sem2/PDS/project/OPP-115/sanitized_policies/"
    path_annotations = "/Users/bhramoss/Documents/CMU/sem2/PDS/project/OPP-115/annotations/"
    regex_html = "(.*).html"
    regex_csv = "(.*).csv"
    for filename_sanitized_policies in os.listdir(path_sanitized_policies):
        if filename_sanitized_policies == ".DS_Store":
            continue
        m = re.match(regex_html, filename_sanitized_policies)
        groups = m.groups()
        filename_sanitized_policies_no_extension = groups[0]
#         print("filename: ", filename_sanitized_policies)
        path_sanitized_policies_file = path_sanitized_policies + "/" + filename_sanitized_policies 
#         file_list = os.listdir(path_annotations)
        f = open(path_sanitized_policies_file, "r")
        content = f.read()
        segments = content.split("|||")
        path_annotations_file = path_annotations + filename_sanitized_policies_no_extension + ".csv"
#         print("path_annotations_file: ", path_annotations_file)
        annotations = pd.read_csv(path_annotations_file, keep_default_na = False, names = ["annotation ID", "batch ID", "annotator ID", "policy ID", "segment ID", "category name", "attribute-value", "policy URL", "date"]) 
        annotations = pd.concat([annotations, pd.get_dummies(annotations['category name'])], axis=1)
#         adding the segments to the dataframe.
        sLength = len(annotations['annotation ID'])
        annotations['segment'] = pd.Series(np.random.randn(sLength), index=annotations.index)
        annotations['segment'] = annotations['segment'].astype('str')
        for i, row in annotations.iterrows():
            segment_id = row['segment ID']
            annotations.at[i, 'segment'] = segments[segment_id]

        dictionary[filename_sanitized_policies_no_extension] = annotations
    return dictionary
        
        
create_df()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [8]:
annotations = create_df()
print(type(annotations))

<class 'dict'>


In [10]:
print(annotations['20_theatlantic.com'])

     annotation ID                          batch ID  annotator ID  policy ID  \
0             2840  test_category_labeling_highlight            84       3635   
1             3616  test_category_labeling_highlight            82       3635   
2             4069  test_category_labeling_highlight            88       3635   
3             2841  test_category_labeling_highlight            84       3635   
4             3617  test_category_labeling_highlight            82       3635   
5             4070  test_category_labeling_highlight            88       3635   
6             2842  test_category_labeling_highlight            84       3635   
7             2843  test_category_labeling_highlight            84       3635   
8             3618  test_category_labeling_highlight            82       3635   
9             3619  test_category_labeling_highlight            82       3635   
10            3620  test_category_labeling_highlight            82       3635   
11            4071  test_cat

Json extraction for unspecified value done below

In [104]:
def parse_api_response(data, key):
    """
    parse Json to get unspecified field count
    
    Args:
        data (string): String of properly formatted JSON.

    Returns:
        
    """
    dictionary = {}
    unspecified_count = 0
    specified_count = 0
    for row in data:
        json_data = json.loads(row)
        for attribute, value in json_data.items():
            for inner_attribute, inner_value in value.items():
#                 print("inner_attribute--->: ", inner_attribute)
#                 == is working, 'is' is not working why ?
                if inner_attribute == "value" and inner_value == "Unspecified":
                    unspecified_count += 1
                else:
                    if inner_attribute == "value":
                        specified_count += 1
    
    print("unspecified_count: ", unspecified_count)
    print("specified_count: ", specified_count)
    dictionary[key] = float(specified_count/(specified_count+unspecified_count))
    print(dictionary)

parse_api_response(annotations['attribute-value'], 'http://www.theatlantic.com/privacy-policy/')

unspecified_count:  687
specified_count:  779
{'http://www.theatlantic.com/privacy-policy/': 0.5313778990450204}
